In [ ]:
from typing import List, Dict, Any, Optional, Protocol, Type
from abc import ABC, abstractmethod
import asyncio
from datetime import datetime
from dataclasses import dataclass
import modelcontextprotocol as mcp
from pydantic import BaseModel, Field

# Tool interface
class Tool(Protocol):
    """Protocol for tool interface."""
    
    async def execute(self, input_data: Any) -> Any:
        """Execute the tool."""
        pass

# Base component
class ToolComponent(ABC):
    """Abstract base class for tool components."""
    
    @abstractmethod
    async def process(self, data: Any) -> Any:
        """Process input data."""
        pass

# Pipeline components
class DataValidator(ToolComponent):
    """Validates input data."""
    
    async def process(self, data: Any) -> Any:
        if not isinstance(data, dict):
            raise ValueError("Input must be a dictionary")
        return data

class DataTransformer(ToolComponent):
    """Transforms input data."""
    
    async def process(self, data: Any) -> Any:
        # Add timestamp
        data["timestamp"] = datetime.now()
        return data

class DataEnricher(ToolComponent):
    """Enriches data with additional information."""
    
    async def process(self, data: Any) -> Any:
        # Add metadata
        data["metadata"] = {
            "version": "1.0",
            "source": "pipeline"
        }
        return data

# Pipeline pattern
class Pipeline:
    """Tool pipeline implementation."""
    
    def __init__(self):
        self.components: List[ToolComponent] = []
        
    def add_component(self, component: ToolComponent) -> None:
        """Add component to pipeline."""
        self.components.append(component)
        
    async def execute(self, data: Any) -> Any:
        """Execute pipeline components in sequence."""
        result = data
        for component in self.components:
            result = await component.process(result)
        return result

# Facade pattern
class ToolFacade:
    """Facade for complex tool interactions."""
    
    def __init__(self):
        self.pipeline = Pipeline()
        self.pipeline.add_component(DataValidator())
        self.pipeline.add_component(DataTransformer())
        self.pipeline.add_component(DataEnricher())
        
    async def process_data(self, data: Any) -> Any:
        """Process data through pipeline."""
        return await self.pipeline.execute(data)

# Adapter pattern
class LegacyTool:
    """Legacy tool with different interface."""
    
    def process_legacy(self, data: str) -> str:
        """Legacy processing method."""
        return f"Processed: {data}"

class LegacyToolAdapter(Tool):
    """Adapter for legacy tool."""
    
    def __init__(self, legacy_tool: LegacyTool):
        self.legacy_tool = legacy_tool
        
    async def execute(self, input_data: Any) -> Any:
        """Adapt legacy tool interface."""
        if isinstance(input_data, dict):
            input_data = str(input_data)
        return self.legacy_tool.process_legacy(input_data)

# Decorator pattern
class LoggingDecorator(Tool):
    """Decorator for adding logging."""
    
    def __init__(self, tool: Tool):
        self.tool = tool
        
    async def execute(self, input_data: Any) -> Any:
        """Add logging to tool execution."""
        print(f"Executing tool with input: {input_data}")
        result = await self.tool.execute(input_data)
        print(f"Tool execution result: {result}")
        return result

class TimingDecorator(Tool):
    """Decorator for timing execution."""
    
    def __init__(self, tool: Tool):
        self.tool = tool
        
    async def execute(self, input_data: Any) -> Any:
        """Add timing to tool execution."""
        start = datetime.now()
        result = await self.tool.execute(input_data)
        duration = (datetime.now() - start).total_seconds()
        print(f"Tool execution took {duration:.3f} seconds")
        return result


In [ ]:
# Example tool implementation
class DataProcessor(Tool):
    """Example data processing tool."""
    
    async def execute(self, input_data: Any) -> Any:
        """Process input data."""
        await asyncio.sleep(0.5)  # Simulate processing
        if isinstance(input_data, dict):
            input_data["processed"] = True
        return input_data

# MCP models
class ProcessRequest(BaseModel):
    """Request for data processing."""
    data: Dict[str, Any] = Field(..., description="Data to process")
    options: Optional[Dict[str, Any]] = Field(default=None, description="Processing options")

class ProcessResponse(BaseModel):
    """Response from data processing."""
    result: Dict[str, Any] = Field(..., description="Processed data")
    metadata: Dict[str, Any] = Field(..., description="Processing metadata")

# Create composite tool
async def create_composite_tool() -> Tool:
    """Create a composite tool with all patterns."""
    
    # Create base tool
    processor = DataProcessor()
    
    # Add decorators
    logged_processor = LoggingDecorator(processor)
    timed_processor = TimingDecorator(logged_processor)
    
    # Create facade
    facade = ToolFacade()
    
    # Create adapter for legacy integration
    legacy_tool = LegacyTool()
    adapter = LegacyToolAdapter(legacy_tool)
    
    # Combine everything into a pipeline
    pipeline = Pipeline()
    pipeline.add_component(DataValidator())
    pipeline.add_component(DataTransformer())
    pipeline.add_component(DataEnricher())
    
    class CompositeTool(Tool):
        async def execute(self, input_data: Any) -> Any:
            # Process through pipeline
            result = await pipeline.execute(input_data)
            
            # Process through decorated tool
            result = await timed_processor.execute(result)
            
            # Process through facade
            result = await facade.process_data(result)
            
            # Process through adapter
            adapter_result = await adapter.execute(result)
            result["legacy_result"] = adapter_result
            
            return result
    
    return CompositeTool()

# Test the composite tool
async def test_composite_tool():
    print("Testing composite tool...")
    
    # Create tool
    tool = await create_composite_tool()
    
    # Create test data
    input_data = {
        "id": "test-1",
        "value": 42,
        "options": {
            "process": True
        }
    }
    
    # Process data
    print("\nProcessing data...")
    result = await tool.execute(input_data)
    
    print("\nFinal result:")
    print(f"ID: {result['id']}")
    print(f"Value: {result['value']}")
    print(f"Timestamp: {result['timestamp']}")
    print(f"Metadata: {result['metadata']}")
    print(f"Legacy Result: {result['legacy_result']}")

# Run test
await test_composite_tool()
